# GLAD alerts module

In [8]:
import ee
import os #à ne pas garder à priori ça ser à rien 
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import HBox, Label
import geemap
import subprocess
import glob
from datetime import datetime
from scripts import gdrive

import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

ee.Initialize()

## fetch the alerts
 
- [x] select a country from the LSIB_SIMPLE 2017 list 
- [x] select a year
- [x] A map will be showing the selected country 
- [x] the downloading on your Gdrive should start
- [x] import these files to your sepal local folder

In [9]:
#const
BUTTON_STYLE = {'button_color':'rgba(28,28,28,.99)'}
bUTTON_LAYOUT = widgets.Layout(
    width='175px',
)
BASEMAP = 'CartoDB.DarkMatter'
CENTER = [4.260202, 15.243700]
NOW = datetime.now()

#template
country_code_field =  widgets.Text(
    value='',
    placeholder='Enter a country code',
)

year_field =  widgets.Dropdown(
    options=['2020','2019'],
    value='2019'
)

output = widgets.Output()

button = widgets.Button(
    description="Use these input!"
)
button.button_style = 'info'
button.style = BUTTON_STYLE
button.layout = bUTTON_LAYOUT

#function
def on_button_clicked(b):
    
    global country_code
    global year
    
    with output:
        button.disabled=True
        country_code_field.disabled=True
        year_field.disabled=True
        
        country_code = country_code_field.value.upper()
        year = year_field.value
        
        display_map(country_code)
        get_alerts(country_code, year)
        task = download_to_disk(country_code, year, alerts)
        #drive_handler.check_status(task)
        
def add_layer_to_map(aoi_ee):
    mapa.clear_layers()
    mapa.add_basemap(BASEMAP)
    mapa.addLayer(aoi, {'color': 'FF0000'})
    mapa.centerObject(aoi, zoom=3)
        
def display_map(country_code):
    global mapa
    global aoi
    
    mapa = geemap.Map(center=CENTER)
    mapa.clear_layers()
    mapa.clear_controls()
    mapa.add_basemap(BASEMAP)
    mapa.layout.height='200px'
    display(mapa)
    
    aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    add_layer_to_map(aoi)
    
def get_alerts(country_code, year):
    """ get the alerts from the GLAD project"""
    global country
    global alerts
    
    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    all_alerts  = ee.ImageCollection('projects/glad/alert/UpdResult')
    alerts = all_alerts.select('conf' + year[-2:]).mosaic().clip(country);
    
def construct_filename(country_code, year):
    """return the filename associated with the current task"""
    
    filename = 'alerts_' + country_code + '_' + year + '_' + NOW.strftime("%Y-%m-%d_%H:%M:%S")
    
    return filename
    
def download_to_disk(country_code, year, alerts):
    """download the tile to the GEE disk"""
    task_config = {
        'image':alerts,
        'description':construct_filename(country_code, year),
        'scale': 30,
        'region':country.geometry(),
        'maxPixels': 1e10
    }
    
    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    return task
    
    
#run
ui1 = HBox([Label('contry code: '), country_code_field])
ui2 = HBox([Label('year: '), year_field])
button.on_click(on_button_clicked)
display(HBox([ui1, ui2, button]))
display(output)


Output()

## Process the glad Alert file
- [x] select the images from the local folder 
- [x] execute the merging command
- [x] execute the clump command 
- [x] execute the calc command

In [10]:
# Create a folder to download the glad images
glad_dir = os.path.join(os.path.expanduser('~'), 'glad_results')+'/'
if not os.path.exists(glad_dir):
    os.makedirs(glad_dir)

alert_map   = glad_dir + "glad_" +year + "_" + country_code + ".tif"
clump_map   = glad_dir + "tmp_clump_" + year + "_" + country_code + ".tif"
alert_stats = glad_dir + "stats_glad_" + year + "_" + country_code + ".txt"
        
#download the files to my folder 
drive_handler = gdrive.gdrive()
files = drive_handler.get_files(construct_filename(country_code, year))
drive_handler.download_files(files, glad_dir)

In [11]:
#postprocess function 
def merge(alert_map, glad_dir):
    """ merge into a single tif files"""
    
    #create command
    command = [
        'gdal_merge.py',
        '-o', alert_map,
        '-v', '-co', '"COMPRESS=LZW"'
    ]
    
    #add the input files
    file_list = []
    for file in glob.glob(glad_dir+"alerts*.tif"):
        command.append(file)

    process = subprocess.run(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    ) 
    
    print(process.stdout)
    
def clump(alert_map, clump_map):
    """ clump the results"""
    process = subprocess.run(
        [
            'oft-clump', 
            '-i', alert_map, 
            '-o', clump_map
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print (process.stdout)
    
def calc(clump_map, alert_map, alert_stats):
    """Compute the statistics per each individual clump"""
    process = subprocess.run(
        [
            'oft-his', 
            '-um', clump_map, 
            '-i', alert_map, 
            '-o', alert_stats,
            '-maxval','3'
        
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print(process.stdout)    

In [12]:
# merge doesn't work with computer smaller than m4 
merge(alert_map, glad_dir)


Processing file     1 of     1,  0.000% completed in 0 minutes.
Filename: /home/prambaud/glad_results/alerts_SZ_2019_2020-06-29_12:21:22.tif
File Size: 16831x7387x1
Pixel Size: 0.000269 x -0.000269
UL:(5.955291,47.809417)   LR:(10.491155,45.818661)
Copy 0,0,16831,7387 to 0,0,16831,7387.



In [13]:
# work like a charm
clump(alert_map, clump_map)

(null) /home/prambaud/glad_results/glad_2019_SZ.tif /home/prambaud/glad_results/tmp_clump_2019_SZ.tif
Driver: GTiff/GeoTIFF
Size is 16831, 7387
Corner Coordinates:
Upper Left  (   5.9552913,  47.8094174) (  5d57'19.05"E, 47d48'33.90"N)
Lower Left  (   5.9552913,  45.8186609) (  5d57'19.05"E, 45d49' 7.18"N)
Upper Right (  10.4911547,  47.8094174) ( 10d29'28.16"E, 47d48'33.90"N)
Lower Right (  10.4911547,  45.8186609) ( 10d29'28.16"E, 45d49' 7.18"N)
Center      (   8.2232230,  46.8140391) (  8d13'23.60"E, 46d48'50.54"N)
...........
...........
Done

DoneClose



In [14]:
# nPixel value 3 exceeds the given max input pixel value 1\nExiting and deleteting empty output
# BUG
calc(clump_map, alert_map, alert_stats)

Pixel Size = (0.00026949,-0.00026949)
Corner Coordinates:
Upper Left  (   5.9552913,  47.8094174) (  5d57'19.05"E, 47d48'33.90"N)
Lower Left  (   5.9552913,  45.8186609) (  5d57'19.05"E, 45d49' 7.18"N)
Upper Right (  10.4911547,  47.8094174) ( 10d29'28.16"E, 47d48'33.90"N)
Lower Right (  10.4911547,  45.8186609) ( 10d29'28.16"E, 45d49' 7.18"N)
Center      (   8.2232230,  46.8140391) (  8d13'23.60"E, 46d48'50.54"N)
Number of input bands is 1
Driver: GTiff/GeoTIFF
Size is 16831, 7387
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]
Segment Min val = 1
Segment Max val = 1
Writing output

DoneClose

